In [1]:
import io
from contextlib import redirect_stdout
from sympy import sympify
from sympy.printing.latex import latex
from IPython.display import display, Math

def parse_cell_variables(offset: int = 0) -> dict:
    """
    Parses the cell history to extract variable names, their corresponding expressions, and values.
    For Pint objects, strips the units and only uses the magnitudes in expressions.

    Args:
        offset (int): The number of previous cells to include for variable extraction. Defaults to 0.

    Returns:
        dict: A dictionary with 'variable_name', 'expression', and 'result' for each variable.
    """
    ipy = get_ipython()
    out = io.StringIO()

    # Capture the history of executed commands
    with redirect_stdout(out):
        ipy.run_line_magic("history", f"{ipy.execution_count - offset}")

    # Get the current variables and their values from the user namespace
    user_ns = ipy.user_ns

    # Extract variable names and expressions from the captured output
    lines = out.getvalue().replace(" ", "").split("\n")

    variables = {}
    for line in lines:
        if "=" in line:
            var_name, expression = line.split("=", 1)
            if var_name in user_ns:
                result = user_ns[var_name]
                variables[var_name] = {
                    'expression': expression,
                    'result': result
                }
    return variables

def replace_numpy_with_sympy(expression):
    numpy_to_sympy = {
        'np.sqrt': 'sp.sqrt',
        'np.sin': 'sp.sin',
        'np.cos': 'sp.cos',
        'np.tan': 'sp.tan',
        'np.exp': 'sp.exp',
        'np.log': 'sp.log',
        'np.array':'Matrix'
        # Add more mappings as needed
    }
    for np_func, sp_func in numpy_to_sympy.items():
        expression = expression.replace(np_func, sp_func)
    return expression

def render_latex_from_variables(variables):
    for var_name, details in variables.items():
        try:
            # Replace NumPy functions with SymPy equivalents
            expression = replace_numpy_with_sympy(details['expression'])
            # Parse and convert to LaTeX
            expr = sympify(expression, evaluate=False)
            var = sympify(var_name)
            result = details['result']
            print(result)
            latex_str = f"{var} = {latex(expr)} = {result}"
            # Display the LaTeX string
            display(Math(latex_str))
        except Exception as e:
            print(f"Error: {e}")

# Example usage
variables = parse_cell_variables()
render_latex_from_variables(variables)


In [2]:
import numpy as np

In [3]:
a = 10
c = np.array([2,5])
if a >20:
    b = 4*c
if a <=20:
    b = 2*c
# Example usage
render_latex_from_variables(parse_cell_variables())



10


<IPython.core.display.Math object>

[2 5]


<IPython.core.display.Math object>

[ 4 10]


<IPython.core.display.Math object>

In [4]:
a = 20*un.m

parse_cell_variables()

NameError: name 'un' is not defined

In [4]:
theta = 10
a = 20 *un.kNm
b = theta * a
put_out()

['theta=10', 'a=20*un.kNm', 'b=theta*a']


$$theta = 10 = 10$$

$$a = 20 \cdot 1 = 20\ \mathrm{kNm}$$

$$b = a \theta = 200\ \mathrm{kNm}$$